# Code to get 3D cells for Unity demo

This code uses the [hra-3d-cell-generation-api](https://github.com/cns-iu/hra-3d-cell-generation-api) to get cell summaries for anatomical structures to be displayed in [ra-cell-type-populations-visualization](https://github.com/cns-iu/hra-cell-type-populations-visualization), which is in turn displayed on the [companion website](https://cns-iu.github.io/hra-cell-type-populations-supporting-information/).

## Import statements

In [81]:
import requests
import json
import csv
import pandas as pd
from io import StringIO

## Generate request params

In [82]:
# Set boilerplat request params
request_params = {
    "file": "https://ccf-ontology.hubmapconsortium.org/objects/v1.2/VH_M_Kidney_L.glb",
    "file_subpath": "VH_M_outer_cortex_of_kidney_L",
    "num_nodes": 2000,
    "node_distribution": {}
}

### Get AS-CT counts from HRA LOD

In [83]:
# Get cell type distribution from HRA LOD
url = 'https://grlc.io/api-git/hubmapconsortium/ccf-grlc/subdir/hra-pop//cell_types_in_anatomical_structurescts_per_as?endpoint=https%3A%2F%2Flod.humanatlas.io%2Fsparql'
header = {"accept": "text/csv"}

# make the GET request and capture text
response = requests.get(url, headers=header)
csv_response = response.text

# convert to data frame
df = pd.read_csv(StringIO(csv_response))

### Create params for hra-3d-cell-generation-api as JSON

In [84]:
# Subset to only get cells for the desired anatomical structure
subset_df = df[((df['as_label'] == 'outer cortex of kidney')) &
               (df['organ'] == 'Left Kidney') &
               (df['sex'] == 'Male') &
               (df['tool'] == 'azimuth')
               ]

# Extract cell labels and percentages for key-value pairs in JSON params
cell_information = subset_df.loc[:, ['cell_label', 'cell_percentage']]

# Create a dictionary where cell_label is the key and cell_percentage is the value
cell_label = cell_information['cell_label'].tolist()
cell_percentage = cell_information['cell_percentage'].tolist()
pairs = dict(zip(cell_label, cell_percentage))

# remove white space in keys to avoid bug. First, let's initialize an empty dictionary to store cleaned key-value pairs
cleaned_cell_label_and_percentage_pairs = {}

# iterate over the original dictionary
for key, value in pairs.items():
  
    # Remove whitespace from the key and store it in the cleaned dictionary
    cleaned_key = key.replace(" ", "")
    
    # thwn assign to cleaned dict
    cleaned_cell_label_and_percentage_pairs[cleaned_key] = value

# add to request params JSON
request_params['node_distribution'] = cleaned_cell_label_and_percentage_pairs

## Fetch 3D cell positions from API and export them to CSV

In [85]:
# set the URL
url = 'https://apfvtab7fp.us-east-2.awsapprunner.com/mesh-3d-cell-population'

# make the POST request 
request = requests.post(url, json=request_params)
response = request.text.split('\n')

# export to CSV
with open('output/3d_cell_distribution.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    for line in response:
        writer.writerow(line.split(','))

['x,y,z,Cell Type', '0.0937562,0.301539,-0.00894732,CorticalThickAscendingLimb', '0.0842288,0.309177,-0.0517078,CorticalThickAscendingLimb', '0.0595087,0.327304,-0.0595075,CorticalThickAscendingLimb', '0.0710523,0.299797,-0.0492607,CorticalThickAscendingLimb', '0.0737688,0.273689,-0.0260053,CorticalThickAscendingLimb', '0.0568878,0.332112,-0.0601199,CorticalThickAscendingLimb', '0.0938507,0.277868,-0.00747833,CorticalThickAscendingLimb', '0.0871762,0.284964,-0.0332848,CorticalThickAscendingLimb', '0.100306,0.291292,-0.0109739,CorticalThickAscendingLimb', '0.0880436,0.26265,-0.00791791,CorticalThickAscendingLimb', '0.0923948,0.292558,-0.00639563,CorticalThickAscendingLimb', '0.0707476,0.32277,-0.0134352,CorticalThickAscendingLimb', '0.0804573,0.259374,-0.0168186,CorticalThickAscendingLimb', '0.0456481,0.352957,-0.0442726,CorticalThickAscendingLimb', '0.093506,0.32093,-0.0365319,CorticalThickAscendingLimb', '0.0925167,0.292297,-0.0420856,CorticalThickAscendingLimb', '0.063263,0.348704,-0